In [1]:
import random 
import logging
import itertools
import time

In [2]:
def problem(N, seed = None): 
    random.seed(seed) 
    return [ list(set(random.randint(0,N-1) for n in  range(random.randint(N //5 ,N//2))))
                 for n in range(random.randint(N,N*5))
    ]

In [4]:
def bypassbranch(subset, i):#bypass a branch 
    for j, sub in reversed( list(enumerate(subset[0:i-1]))) :
    
        if sub == 0:
            subset[j] = 1
            return subset, j+1

    return subset, 0

def nextvertex(subset, i, m):
    if i < m:
        subset[i] = 0
        return subset, i+1
    else:
        for j, sub in reversed( list(enumerate(subset[0:m-1]))) : 
            if sub == 0:
                subset[j] = 1
                return subset, j+1
                
    return subset, 0

In [5]:
def BB(N):
    goal =  set(range(N))
    sets = problem(N,seed =42) 
    costs = list( len(s) for s  in sets)
    subset = [1 for x in range(len(sets))]#all sets in
    subset[0] = 0
    bestCost = sum(costs) #actually the worst cost 
    i = 1
    Nodes = list()
    Nodes.append(subset)
    z=time.time()
    solution = list()
    while i > 0:

        if i < len(sets):
            Nodes.append(subset) 
            cost, temp_Set = 0, set()# temp for temporary
            for k , (sub , c) in enumerate(zip(subset[0:i],costs[0:i])):
                cost += sub*c   #if 1 adds the cost to total
                if sub == 1: temp_Set.update(set(sets[k]))#if 1 add the set to the cover

            if cost > bestCost:#if the cost is larger than the currently best one, we can bypass the branch 
                subset, i = bypassbranch(subset, i)
                continue
            temp_Set.update(set(itertools.chain(*sets[i::])))
            if temp_Set != goal:#that means that the set was essential at this point to complete the uni.
                subset, i = bypassbranch(subset, i)
            else:
                subset, i = nextvertex(subset, i, len(sets))
        
        else: 
            
            cost , final_Set = 0, set() 
            for k , (sub , c) in enumerate(zip(subset,costs)):
                cost+= sub*c
                if sub == 1 :final_Set.update(set(sets[k]))
            if cost < bestCost and final_Set == goal: 
                bestCost = cost 
                bestSubset = subset[:]
                print(bestCost)
            subset , i = nextvertex(subset , i , len(sets))
        if (time.time()-z > 900.0):
            i = 0
    for x, sub in enumerate(bestSubset):
        if sub==1:
            solution.append(sets[x])
    unique_data = [list(i) for i  in set(tuple(i) for i in Nodes)]
    logging.info(
        f"Branch and Bound solution for N={N}: Cost ={bestCost}  / visited Nodes={len(Nodes)} / Search time= {time.time()-z} (bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)"
    )
    logging.debug(f"{solution}")
    

In [6]:
logging.getLogger().setLevel(logging.INFO)
for N in [5, 10, 20, 100, 500, 1000]:
    BB(N)

6
5


INFO:root:Branch and Bound solution for N=5: Cost =5  / visited Nodes=115458 / Search time= 0.4110400676727295 (bloat=0%)


12
11
10


INFO:root:Branch and Bound solution for N=10: Cost =10  / visited Nodes=957418 / Search time= 4.948209047317505 (bloat=0%)


32
31
28
27
24


INFO:root:Branch and Bound solution for N=20: Cost =23  / visited Nodes=101999 / Search time= 0.46616697311401367 (bloat=15%)


23
275
259
258
256
254
240
229
221


KeyboardInterrupt: 